1) The first step in the process, is to select the data.  In this case, there is just one data source (movie reviews).  These are presented to the UI by asking CloudSearch to tell us the available domains.  In this case, it is just rocket (movie reviews)


In [4]:
import json
import boto3
cs_client = boto3.client('cloudsearch',)
cs_client.list_domain_names()['DomainNames']

{'rocket': '2013-01-01'}

In [5]:
 # list data sources
cloudsearch_client = boto3.client('cloudsearchdomain',endpoint_url='https://search-rocket-v3wsrkhqhk3ls5u32q2s7jsldq.us-east-1.cloudsearch.amazonaws.com')
cloudsearch_client.search(
    #cursor='string',
    #expr='string',
    #facet='string',
    #filterQuery='string',
    #highlight='string',
    #partial=True|False,
    query='James Bond',
    #queryOptions='string',
    #queryParser='simple'|'structured'|'lucene'|'dismax',
    #returnFields='string',
    size=10,
    #sort='string',
    #start=123,
    #stats='string'
)

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json; charset=UTF-8',
   'content-length': '15090',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'status': {'timems': 28, 'rid': 'gL+d9YMthwgKUb2O'},
 'hits': {'found': 179,
  'start': 0,
  'hit': [{'id': 'imdb-reviews/pos1/8523_10.txt',
    'fields': {'content': ['In 1983 two Bond movies was made  one was the official Bond movie Octopussy starring Roger Moore who starred as James Bond for the first time in Live and Let Die and the other was the unofficial Bond movie Never Say Never Again starring Sean Connery who played the role as James Bond for the first time in Dr. No  that film was also the first 007 James Bond movie to be made. Never Say Never Again is called unofficial because the company that made the other James Bond movies didn t make this one. Never Say Never Again is also a remake of the 1965 007 movie Thunderball  there are several differences in Never Say Never Again tha

2) Next, we generate concepts.  The following code blocks show the application's CRUD operations for Concepts on AWS.  For movies, we are considering sentiment. Positive (1) and negative (0).

Services in RoCKET are available via an API that interacts with S3, DynamoDB, CloudSearch, and an EKS compute cluster.  All functionality is has controlled access through the AWS API Gateway services, depending on the user's role.

In [7]:
# hit the services directly (ping test)
import requests

# test get concept
headers = {
    'cache-control': 'no-cache',
    'x-api-key': 'LOw5yIHqzf9c5layPN0SNaNXgTOWzH8I5z7y5V6f',
    'Host': 'nz33f5gf45.execute-api.us-east-1.amazonaws.com'
}

path = r'https://vpce-02c9693a7afa5a3d4-d490t8kz.execute-api.us-east-1.vpce.amazonaws.com/dev/'

r = requests.get(path+'ping', headers=headers)
r.json()


'pong'

In [8]:
# get the concepts
r = requests.get(path+'getConcepts', headers=headers)
r.json()

[{'keywords': 'Active Learning, Enrichment, Concepts,Fun,enrich,enriched,relearn,concepts,concept,labeling,enrichment,learn,memorize,labels,learning,relearning,understand',
  'concept_name': 'RoCKET'},
 {'keywords': 'what,what a b', 'concept_name': 'biggieData'}]

In [9]:
# create sentiment concept
parameters = {
    'concept_name':'positive sentiment',
    'keywords':'great, fantastic'
}
r = requests.get(path+'createConcept', params=parameters, headers=headers)
r.json()

[{'keywords': 'great, fantastic', 'concept_name': 'positive sentiment'},
 {'keywords': 'Active Learning, Enrichment, Concepts,Fun,enrich,enriched,relearn,concepts,concept,labeling,enrichment,learn,memorize,labels,learning,relearning,understand',
  'concept_name': 'RoCKET'},
 {'keywords': 'what,what a b', 'concept_name': 'biggieData'}]

In [12]:
# update sentiment concept
parameters = {
    'concept_name':'positive sentiment',
    'keywords':'great,fantastic,super,10 of 10'
}
r = requests.get(path+'updateConcept', params=parameters, headers=headers)
r.json()

[{'keywords': 'great,fantastic,super,10 of 10',
  'concept_name': 'positive sentiment'},
 {'keywords': 'Active Learning, Enrichment, Concepts,Fun,enrich,enriched,relearn,concepts,concept,labeling,enrichment,learn,memorize,labels,learning,relearning,understand',
  'concept_name': 'RoCKET'}]

In [13]:
# test delete concept
parameters = {
    'concept_name':'RoCKET'
}
r = requests.get(path+'deleteConcept', params=parameters, headers=headers)
r.json()

[{'keywords': 'great,fantastic,super,10 of 10',
  'concept_name': 'positive sentiment'}]

3) With the sentiment concept defined, we then enrich it using related words.


In [2]:
import pandas as pd
pos_df = pd.read_csv('/Users/rwl012/Downloads/Batch_3494319_batch_results.csv')
neg_df = pd.read_csv('/Users/rwl012/Downloads/Batch_3494476_batch_results.csv')

In [3]:
pos_df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.text',
       'Answer.highlights', 'Approve', 'Reject'],
      dtype='object')

In [4]:
print(len(pos_df))
print(len(neg_df))

29080
22565


In [5]:
# only get those accepted records
pos_df_approved = pos_df[pos_df['AssignmentStatus']=='Approved']
neg_df_approved = neg_df[neg_df['AssignmentStatus']=='Approved']

In [6]:
print(len(pos_df_approved))
print(len(neg_df_approved))

19108
18763


In [7]:
# buid the dataset
import random
import numpy as np
labeled_pos = random.sample(pos_df_approved['Answer.highlights'].tolist(), 500)
labeled_neg = random.sample(neg_df_approved['Answer.highlights'].tolist(), 500)
Y = np.concatenate([np.zeros(500),np.ones(500)])
X = (labeled_neg + labeled_pos)

In [44]:
#load pretrained model
import pymagnitude
pretrained_magnitude = r'/Users/rwl012/Downloads/GoogleNews-vectors-negative300.magnitude'

vectors = pymagnitude.Magnitude(pretrained_magnitude)


def get_document_features(data_in,highlights=False,c=0.5):
    data_in = data_in.replace('<span class=\"active_text\">', '')
    data_in = data_in.replace('</span>', '')
    body = data_in.split(r'\n                                    ')[1]
    
    body = body.replace('\n', '')
    vecs = vectors.query(body.split(' '))
    avg_vec = np.mean(vecs, axis=(0))
    
    if highlights == False:
        high_text = data_in.split(r'\n                                    ')[0]
        high_text = high_text.replace('\n', '')
        high_vecs = vectors.query(high_text.split(' '))
        avg_highvec = np.mean(high_vecs, axis=(0))
        return avg_highvec *(c) + (1-c)*avg_vec
    else:
        return avg_vec


In [8]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [48]:
# # populate the testing features
# X_test_reg_features = list()
# X_test_high_features = list()

# for idx,text in enumerate(X_test):
#     X_test_reg_features.append(get_document_features(text))
#     X_test_high_features.append(get_document_features(text,True,0.5))


In [51]:
import pickle
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('Y_train.pkl', 'wb') as f:
    pickle.dump(Y_train, f)
with open('Y_test.pkl', 'wb') as f:
    pickle.dump(Y_test, f)

In [50]:
# lets save these

with open('X_test_reg_features.pkl', 'wb') as f:
    pickle.dump(X_test_reg_features, f)
with open('X_test_high_features.pkl', 'wb') as f:
    pickle.dump(X_test_high_features, f)


In [73]:
print(Y_test.shape)

(200,)


In [76]:
reg_predictions[0]

array([0.50394519, 0.49605481])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

clf = LogisticRegression()

y_train_list = list()
norm_feat_train_list = list()
high_feat_train_list = list()

norm_accuracy = list()
high_accuracy = list()

# simulated active learning
for idx,text in enumerate(X_train):
    norm_feat_train_list.append(get_document_features(text))
    high_feat_train_list.append(get_document_features(text,True,0.5))
    y_train_list.append(Y_train[idx])
    
    # if enough classes to fit
    if len(set(y_train_list)) > 1:
        reg_model = clf.fit(np.vstack(norm_feat_train_list), y_train_list)
        high_model = clf.fit(np.vstack(high_feat_train_list), y_train_list)

        reg_predictions = reg_model.predict_proba(X_test_reg_features)
        high_predictions = high_model.predict_proba(X_test_high_features)

        nacc = accuracy_score( np.where(reg_predictions[:,0] > 0.5, 1, 0), Y_test, normalize=False)
        hacc = accuracy_score( np.where(high_predictions[:,0]> 0.5, 1, 0), Y_test, normalize=False)
        
        print(nacc )
        print(hacc)
        norm_accuracy.append(nacc)
        high_accuracy.append(hacc)

89
91
106
106
105
98
94
94
94
94
94
94
94
94
94
94
94
94
94
94
80
84
94
94
76
80
106
106
80
93
94
94
80
90
94
94
77
87
94
94
94
94
94
94
74
87
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
50
63
106
106
89
84
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
96
93
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
93
80
84
54
71
86
78
94
93
76
80
94
93
63
82
93
94
94
94
94
91
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
95
92
94
93
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
94
92
94
85
94
91
94
8

In [77]:
print(type(reg_predictions))

<class 'numpy.ndarray'>


In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [42]:
# just the response
from tensorboardX import SummaryWriter
# Reference/Credit Stats class & reporting mechanisms: danielnbarbosa/drlnd_collaboration_and_competition 
class Stats():
    def __init__(self):
        self.score = None
        self.avg_score = None
        self.std_dev = None
        self.scores = []                         # list containing scores from each episode
        self.avg_scores = []                     # list containing average scores after each episode
        self.scores_window = deque(maxlen=100)   # last 100 scores
        self.best_avg_score = -np.Inf            # best score for a single episode
        self.time_start = time.time()            # track cumulative wall time
        self.total_steps = 0                     # track cumulative steps taken
        self.writer = SummaryWriter()

    def update(self, steps, rewards, i_episode):
        """Update stats after each episode."""
        self.total_steps += steps
        self.score = sum(rewards)
        self.scores_window.append(self.score)
        self.scores.append(self.score)
        self.avg_score = np.mean(self.scores_window)
        self.avg_scores.append(self.avg_score)
        self.std_dev = np.std(self.scores_window)
        # update best average score
        if self.avg_score > self.best_avg_score and i_episode > 100:
            self.best_avg_score = self.avg_score

    def is_solved(self, i_episode, solve_score):
        """Define solve criteria."""
        return self.avg_score >= solve_score and i_episode >= 100

    def print_episode(self, i_episode, steps, stats_format, buffer_len, noise_weight,
                      critic_loss_01, critic_loss_02,
                      actor_loss_01, actor_loss_02,
                      noise_val_01, noise_val_02,
                      rewards_01, rewards_02):
        common_stats = 'Episode: {:5}   Avg: {:8.3f}   BestAvg: {:8.3f}   σ: {:8.3f}  |  Steps: {:8}   Reward: {:8.3f}  |  '.format(i_episode, self.avg_score, self.best_avg_score, self.std_dev, steps, self.score)
        print('\r' + common_stats + stats_format.format(buffer_len, noise_weight), end="")
        # log lots of stuff to tensorboard
        self.writer.add_scalar('global/reward', self.score, i_episode)
        self.writer.add_scalar('global/std_dev', self.std_dev, i_episode)
        self.writer.add_scalar('global/avg_reward', self.avg_score, i_episode)
        self.writer.add_scalar('global/buffer_len', buffer_len, i_episode)
        self.writer.add_scalar('global/noise_weight', noise_weight, i_episode)
        self.writer.add_scalar('agent_01/critic_loss', critic_loss_01, i_episode)
        self.writer.add_scalar('agent_02/critic_loss', critic_loss_02, i_episode)
        self.writer.add_scalar('agent_01/actor_loss', actor_loss_01, i_episode)
        self.writer.add_scalar('agent_02/actor_loss', actor_loss_02, i_episode)
        self.writer.add_scalar('agent_01/noise_val_01', noise_val_01[0], i_episode)
        self.writer.add_scalar('agent_01/noise_val_02', noise_val_01[1], i_episode)
        self.writer.add_scalar('agent_02/noise_val_01', noise_val_02[0], i_episode)
        self.writer.add_scalar('agent_02/noise_val_02', noise_val_02[1], i_episode)
        self.writer.add_scalar('agent_01/reward', rewards_01, i_episode)
        self.writer.add_scalar('agent_02/reward', rewards_02, i_episode)

    def print_epoch(self, i_episode, stats_format, *args):
        n_secs = int(time.time() - self.time_start)
        common_stats = 'Episode: {:5}   Avg: {:8.3f}   BestAvg: {:8.3f}   σ: {:8.3f}  |  Steps: {:8}   Secs: {:6}      |  '.format(i_episode, self.avg_score, self.best_avg_score, self.std_dev, self.total_steps, n_secs)
        print('\r' + common_stats + stats_format.format(*args))

    def print_solve(self, i_episode, stats_format, *args):
        self.print_epoch(i_episode, stats_format, *args)
        print('\nSolved in {:d} episodes!'.format(i_episode-100))


In [ ]:
# list containing scores from each episode
stats = Stats()
stats_format = 'Buffer: {:6}   NoiseW: {:.4}'

keep_going = True:
while keep_going:
    # label pos/neg 
    
    # train logistic regression model
    
    # score using next closest
    
    # print results
#     stats.update(t, [np.max(per_agent_rewards)], i_episode)  # use max over all agents as episode reward
#     stats.print_episode(i_episode, t, stats_format, buffer_len, self.agent.noise_weight,
#                         self.agent.agents[0].critic_loss, self.agent.agents[1].critic_loss,
#                         self.agent.agents[0].actor_loss, self.agent.agents[1].actor_loss,
#                         self.agent.agents[0].noise_val, self.agent.agents[1].noise_val,
#                         per_agent_rewards[0], per_agent_rewards[1])
    
    # check if done

        

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

plt.plot(predictions)
plt.title('prediction plots')
plt.show()